In [1]:
# підключення базових модулей
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
import shutil
from pprint import pprint

# параметри виведення
pd.set_option("display.max_columns", 500) # кількість колонок
pd.set_option("display.max_rows", 1000) # кількість рядків
pd.set_option("display.max_colwidth", 300) # ширина колонок
pd.set_option("display.precision", 5) # кількість знаків після коми

# вимикаємо зайві попередження
import warnings
warnings.filterwarnings("ignore")

# друк всіх результатів в одній комірці а не тільки останнього
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# магічний метод для того щоб отримувати графіки біля комірок з кодом
%matplotlib inline

In [ ]:
import random
import math


# Визначення функції Сфери
def sphere_function(x):
    return sum(xi ** 2 for xi in x)


# Hill Climbing
def hill_climbing(func, bounds, iterations=1000, epsilon=1e-6):
    # Ініціалізація випадкової точки
    current_point = [random.uniform(b[0], b[1]) for b in bounds]
    current_value = func(current_point)
    # Розмір кроку для генерації сусідів - розраховуємо такий, щоб при найгіршому випадку бути в змозі пройтися від краю до краю вздовж найдовшої сторони
    step_size = (max((b[1] - b[0]) for b in bounds) / iterations)

    for _ in range(iterations):

        # Генерація сусідів
        x, y = current_point
        neighbors = [
            (x + step_size, y),
            (x - step_size, y),
            (x, y + step_size),
            (x, y - step_size),
        ]

        # Пошук найкращого сусіда
        next_point = None
        next_value = np.inf

        for neighbor in neighbors:
            value = func(neighbor)
            if value < next_value:
                next_point = neighbor
                next_value = value

        # Якщо покращення результату менше епсілон — зупиняємось  
        if epsilon >= current_value - next_value:
            break

        # Переходимо до кращого сусіда
        current_point, current_value = next_point, next_value

    return current_point, current_value


# Random Local Search
def random_local_search(func, bounds, iterations=1000, epsilon=1e-6):

    PROBABILITY = 0.2 # ймовірність переходу до гіршого сусіда
    ACURACY_RATIO = 2 # коефіцієнт точності (враховує, що випадкові з рівною ймовірністю 
                        #можуть йти проти напрямку оптимізації, отже треба збільшити крок, щоб обійти весь простір рішень)
    EARLY_STOP_LIMIT = 20 # ліміт ранньої зупинки

    # Ініціалізація випадкової точки
    current_point = [random.uniform(b[0], b[1]) for b in bounds]
    current_value = func(current_point)
    # Розмір кроку для генерації сусідів - розраховуємо такий, щоб при найгіршому випадку 
    # бути в змозі пройтися від краю до краю вздовж найдовшої сторони з увахуванням ймовірностей йти до гіршого сусіда
    step_size = max((b[1] - b[0]) for b in bounds) / iterations / (1 - PROBABILITY) * ACURACY_RATIO

    early_stop_counter = 0
    for iteration in range(iterations):
        
        # Отримання випадкового сусіда
        x, y = current_point
        new_x = x + random.uniform(-step_size, step_size)
        new_y = y + random.uniform(-step_size, step_size)
        new_point = (new_x, new_y)
        new_value = func(new_point)

        # Якщо покращення результату менше епсілон — запускаємо лічильник ранньої зупинки
        if epsilon >= current_value - new_value:
            early_stop_counter += 1
        else:
            # Якщо покращення результату більше епсілон — скидаємо лічильник ранньої зупинки
            early_stop_counter = 0

        # Якщо лічильник ранньої зупинки досягнув ліміту — зупиняємось
        if early_stop_counter >= EARLY_STOP_LIMIT:
            break

        # Перевірка умови переходу
        if new_value < current_value or random.random() < PROBABILITY:
            current_point, current_value = new_point, new_value

    # print(f"Iteration {iteration}, Step size: {step_size}, Current value: {current_value}")
    return current_point, current_value


# Simulated Annealing
def simulated_annealing(func, bounds, iterations=1000, temp=1000, cooling_rate=0.95, epsilon=1e-6):
    # Ініціалізація випадкової точки
    current_solution = [random.uniform(b[0], b[1]) for b in bounds]
    current_energy = func(current_solution)

    iteration_counter = 0 # лічильник ітерацій
    while temp > epsilon or iteration_counter < iterations:
        iteration_counter += 1
        
        # Генерація нового рішення
        x, y = current_solution
        new_x = x + random.uniform(-1, 1)
        new_y = y + random.uniform(-1, 1)
        new_solution = (new_x, new_y)

        new_energy = func(new_solution)
        delta_energy = new_energy - current_energy

        if delta_energy < 0 or random.random() < math.exp(-delta_energy / temp):
            current_solution = new_solution
            current_energy = new_energy

        temp *= cooling_rate

    # print(f"Iteration {iteration_counter}, Step size: {temp}, Current value: {current_solution}")
    return current_solution, current_energy


if __name__ == "__main__":
    # Межі для функції
    bounds = [(-5, 5), (-5, 5)]

    # Виконання алгоритмів
    print("Hill Climbing:")
    hc_solution, hc_value = hill_climbing(sphere_function, bounds)
    print("Розв'язок:", hc_solution, "Значення:", hc_value)

    print("\nRandom Local Search:")
    rls_solution, rls_value = random_local_search(sphere_function, bounds)
    print("Розв'язок:", rls_solution, "Значення:", rls_value)

    print("\nSimulated Annealing:")
    sa_solution, sa_value = simulated_annealing(sphere_function, bounds)
    print("Розв'язок:", sa_solution, "Значення:", sa_value)

Hill Climbing:
Розв'язок: (-0.0012485618908738318, 0.0013914423296396382) Значення: 3.4950185520554217e-06

Random Local Search:
Розв'язок: (0.017796386350811738, 0.0012946229796052533) Значення: 0.0003183874158066803

Simulated Annealing:
Розв'язок: (0.02393637485227429, -0.009589402764935473) Значення: 0.0006649066864567411
